In [2]:
from xgboost import XGBClassifier
from xgboost import plot_importance
from matplotlib import pyplot

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from numpy import sort
from sklearn.feature_selection import SelectFromModel

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from itertools import combinations

In [18]:
df = pd.read_csv("DL-Train.csv")
#df = df.drop(["year","name","college","position"],1)
df.head()
# 1009

,year,name,college,position,height_in,weight_lbs,40yards,benchpress,vert_leap_in,broad_jump_in,shuttle,3cone,hasbling,allamerican,hasawards,drafted
0,1997,duane ashman,Virginia,DE,75.5,274,5.03,19.0,32.5,110.0,4.83,8.80,0,0,0,Y
1,1997,bert berry,Notre Dame,DE,74.6,243,4.64,27.0,37.5,123.0,4.49,7.79,0,0,0,N
2,1997,peter boulware,Florida State,DE,76.1,254,4.76,NaN,34.0,118.0,4.20,7.64,1,1,1,Y
3,1997,cornell brown,Virginia Tech,DE,73.1,244,5.00,19.0,32.0,115.0,4.33,7.81,1,1,1,Y
4,1997,william carr,Michigan,DT,72.1,302,5.41,19.0,25.5,83.0,5.14,8.42,0,0,0,Y


In [7]:
from splinter import Browser
from sys import platform
import os
import csv
import re

executable_path = None

if platform == "linux" or platform == "linux2":
    executable_path = {'executable_path': os.path.join("binaries", "chromedriver-linux")}
elif platform == "darwin":
    executable_path = {'executable_path': os.path.join("binaries", "chromedriver-mac")}
elif platform == "win32":
    executable_path = {'executable_path': os.path.join("..","..","binaries", "chromedriver-windows.exe")}

browser = Browser('chrome', **executable_path, headless=False)

In [15]:
for index, row in df.iterrows():
    if (df.loc[index,"hasbling"] != 1.0):
        if (df.loc[index,"hasbling"] != 0.0):
            x = row["name"]
            browser.visit(f"https://www.sports-reference.com/cfb/search/search.fcgi?hint=&search={x}")
            if(len(browser.find_by_css("h1[itemprop='name']")) == 1):
                if (len(browser.find_by_css("ul#bling")) > 0):
                    df.loc[index,"hasbling"] = 1
                else:
                    df.loc[index,"hasbling"] = 0
                if (len(browser.find_by_css("li.bling_all_star")) > 0):
                    df.loc[index,"allamerican"] = 1
                else:
                    df.loc[index,"allamerican"] = 0
                if (len(browser.find_by_css("li.bling_hof")) > 0):
                    df.loc[index,"heisman"] = 1
                else:
                    df.loc[index,"heisman"] = 0
                if (len(browser.find_by_css("div#leaderboard_awards")) > 0):
                    df.loc[index,"hasawards"] = 1
                else:
                    df.loc[index,"hasawards"] = 0
            elif (len(browser.find_by_css("div.search-item")) == 0):
                df.loc[index,"hasbling"] = 0
                df.loc[index,"allamerican"] = 0
                df.loc[index,"heisman"] = 0
                df.loc[index,"hasawards"] = 0
            else:
                listed = browser.find_by_css("div.search-item")

                for x in listed:
                    if (re.sub(r'\(.*\)', "", row["college"]).strip() in x.find_by_css("div.search-item-league").text):
                        x.find_by_css("div.search-item-name a").click()
                        if (len(browser.find_by_css("h1[itemprop='name']")) == 1):
                            if (len(browser.find_by_css("ul#bling")) > 0):
                                df.loc[index,"hasbling"] = 1
                            else:
                                df.loc[index,"hasbling"] = 0
                            if (len(browser.find_by_css("li.bling_all_star")) > 0):
                                df.loc[index,"allamerican"] = 1
                            else:
                                df.loc[index,"allamerican"] = 0
                            if (len(browser.find_by_css("li.bling_hof")) > 0):
                                df.loc[index,"heisman"] = 1
                            else:
                                df.loc[index,"heisman"] = 0
                            if (len(browser.find_by_css("div#leaderboard_awards")) > 0):
                                df.loc[index,"hasawards"] = 1
                            else:
                                df.loc[index,"hasawards"] = 0
                            break
                        else:
                            df.loc[index,"hasbling"] = 0
                            df.loc[index,"allamerican"] = 0
                            df.loc[index,"heisman"] = 0
                            df.loc[index,"hasawards"] = 0
                            break
                    else:
                        df.loc[index,"hasbling"] = 0
                        df.loc[index,"allamerican"] = 0
                        df.loc[index,"heisman"] = 0
                        df.loc[index,"hasawards"] = 0

In [16]:
df.to_csv("DL-Train.csv",index=False)